実行環境 <br>
- Python 3.11.3
- Pandas 2.0.2
- Numpy 1.24.3

In [1]:
import pandas as pd
import numpy as np

# 費用情報

In [86]:
cost_info = pd.read_csv('./費用情報.csv',dtype='object')
cost_info_0 = cost_info.copy()

In [87]:
cost_info = cost_info_0.copy()

# '○万円' -> int の変換
def obj_to_int(col):
  cost_info[col].replace('-','000', inplace=True)
  cost_info[col] = cost_info[col].str[:-2].astype(float)
  cost_info[col] = cost_info[col]*10000
  cost_info[col] = cost_info[col].astype(pd.Int64Dtype())
  cost_info[col].replace(0,np.NAN, inplace=True)
  return cost_info

# 賃料：object -> int
cost_info = obj_to_int('賃料')

# 敷金 object -> int
cost_info = obj_to_int('敷金')

# 礼金 object -> int
cost_info = obj_to_int('礼金')

# 保証金 object -> int
cost_info = obj_to_int('保証金')

# 敷引・償却 object -> int
cost_info = obj_to_int('敷引・償却')

# 管理費 object -> int 
cost_info['管理費'] = cost_info['管理費'].str[:-1].astype(int)


cost_info.head()

,物件CD,賃料,管理費,敷金,礼金,保証金,敷引・償却
0,100343313667,69000,10000,<NA>,69000,<NA>,<NA>
1,100343314105,69000,10000,<NA>,69000,<NA>,<NA>
2,100343313391,70000,10000,<NA>,70000,<NA>,<NA>
3,100343842510,70000,5000,70000,<NA>,<NA>,<NA>
4,100343709215,70000,5000,70000,<NA>,<NA>,<NA>


# 設備情報

In [40]:
facility_info = pd.read_csv('./設備情報.csv')
facility_info_0 = facility_info.copy()

In [83]:
facility_info = facility_info_0.copy()
import warnings
warnings.filterwarnings('ignore')

features = set()
for options in facility_info['オプション']:
    features.update(options.split('、'))

for feature in features:
    facility_info[feature] = facility_info['オプション'].apply(lambda x: 1 if feature in x else 0)

facility_info.head()


,物件CD,オプション,CATVインターネット,即入居可,シャッター,角住戸,単身者相談,カーポート,エアコン全室,上階無し,...,グリル付,BS・CS,耐震ラッチ吊戸棚,照明付,3駅以上利用可,クロゼット,メゾネット,3沿線以上利用可,防犯カメラ,二重床構造
0,100343313667,バストイレ別、バルコニー、エアコン、ガスコンロ対応、クロゼット、フローリング、TVインターホ...,0,0,0,0,1,0,1,0,...,0,1,0,1,1,1,0,1,1,0
1,100343314105,バストイレ別、バルコニー、エアコン、ガスコンロ対応、クロゼット、フローリング、TVインターホ...,0,0,0,0,1,0,1,0,...,0,1,0,1,1,1,0,1,1,0
2,100343313391,バストイレ別、バルコニー、エアコン、ガスコンロ対応、クロゼット、フローリング、TVインターホ...,0,0,0,0,1,0,1,0,...,0,1,0,1,1,1,0,1,1,0
3,100343842510,バストイレ別、エアコン、フローリング、TVインターホン、浴室乾燥機、オートロック、室内洗濯置...,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,100343709215,バストイレ別、エアコン、フローリング、TVインターホン、浴室乾燥機、オートロック、室内洗濯置...,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


# 物件情報

In [93]:
property_info = pd.read_csv('./物件情報.csv')
property_info_0 = property_info.copy()

In [100]:
property_info = property_info_0.copy()
property_info['リンク'] = property_info['リンク'].apply(lambda x: 'https://suumo.jp' + x)
property_info.head()

,物件名,リンク
0,リヴシティ横浜東ベイサイド,https://suumo.jp/chintai/bc_100343313667/
1,リヴシティ横浜東ベイサイド,https://suumo.jp/chintai/bc_100343314105/
2,リヴシティ横浜東ベイサイド,https://suumo.jp/chintai/bc_100343313391/
3,マリーナヨコハマベイサイド 203号室,https://suumo.jp/chintai/bc_100343842510/
4,マリーナヨコハマベイサイド 203号室,https://suumo.jp/chintai/bc_100343709215/


# 物件基本情報 <br>
- 間取り詳細をどうするか検討。一旦そのまま

In [4]:
property_basic_info = pd.read_csv('./物件基本情報.csv',dtype='object')
property_basic_info_0 = property_basic_info.copy()

In [82]:
property_basic_info = property_basic_info_0.copy()

# 物件名 -> 物件名, 不動産会社
property_basic_info['物件名'] = property_basic_info['物件名'].str.split(' - ')
property_basic_info['不動産'] = property_basic_info['物件名'].str[1]
property_basic_info['不動産'] = property_basic_info['不動産'].str.replace('が提供する賃貸物件情報','')
property_basic_info['物件名'] = property_basic_info['物件名'].str[0]

# 所在地を 都道府県, 市区町村, 番地 に分割
def div_plc(a,b,c,d):
  div_col = property_basic_info['所在地'].apply(
    lambda x: 
      x[:x.find(a)+1] if x.find(a) > 0 
      else x[:x.find(b)+1] if x.find(b) > 0 
      else x[:x.find(c)+1] if x.find(c) > 0 
      else x[:x.find(d)+1]
  )
  return div_col
# 都道府県
property_basic_info['所在地_都道府県'] = div_plc('都','道','府','県')
# 市区町村
property_basic_info['所在地'] = property_basic_info.apply(
  lambda x:
    x['所在地'].replace(x['所在地_都道府県'],'') ,axis=1
  )
property_basic_info['所在地_市区町村'] = div_plc('市','区','町','村')
# 番地
property_basic_info['所在地_番地'] = property_basic_info.apply(
  lambda x:
    x['所在地'].replace(x['所在地_市区町村'],'') ,axis=1
  )
# 処理前の列を削除
property_basic_info.drop(columns='所在地',inplace=True)

# 階建 
floor_story = property_basic_info['階建'].str.split('/')
property_basic_info['階'] = floor_story.str[0].str[:-1].astype(int)
property_basic_info['階建'] = floor_story.str[1].str[:-2].astype(int)

# 専有面積
property_basic_info['専有面積'] = property_basic_info['専有面積'].str[:-2].astype(float)
property_basic_info = property_basic_info.rename(columns={'専有面積':'専有面積(m^2)'})

# 更新日
property_basic_info['更新日'] = pd.to_datetime(property_basic_info['更新日'])


property_basic_info = property_basic_info[[
  '物件CD', '物件名','築年数', '所在地_都道府県', '所在地_市区町村', '所在地_番地',
  '建築種別', '構造', '向き', '階建', '階', '間取り', '間取り詳細', '専有面積(m^2)', '間取り図',
  '取引様態', '取引先', '更新日'
  ]]

property_basic_info.head()

,物件CD,物件名,築年数,所在地_都道府県,所在地_市区町村,所在地_番地,建築種別,構造,向き,階建,階,間取り,間取り詳細,専有面積(m^2),間取り図,取引様態,取引先,更新日
0,100343313667,リヴシティ横浜東ベイサイド,2010年11月,神奈川県,横浜市,神奈川区東神奈川２,マンション,鉄筋コン,南東,10,6,1K,洋6.7,20.02,https://img01.suumo.com/front/gazo/fr/bukken/6...,仲介,(株)タウンハウジング神奈川 大倉山店,2023-09-12
1,100343314105,リヴシティ横浜東ベイサイド,2010年11月,神奈川県,横浜市,神奈川区東神奈川２,マンション,鉄筋コン,南東,10,6,1K,洋6.7,20.02,https://img01.suumo.com/front/gazo/fr/bukken/1...,仲介,(株)タウンハウジング神奈川 鶴見店,2023-09-13
2,100343313391,リヴシティ横浜東ベイサイド,2010年11月,神奈川県,横浜市,神奈川区東神奈川２,マンション,鉄筋コン,南東,10,6,1K,-,20.02,https://img01.suumo.com/front/gazo/fr/bukken/3...,仲介,(株)タウンハウジング神奈川 横浜店,2023-09-12
3,100343842510,マリーナヨコハマベイサイド 203号室,2017年6月,神奈川県,横浜市,神奈川区東神奈川２,マンション,鉄筋コン,南東,5,2,1K,洋6.7,20.18,https://img01.suumo.com/front/gazo/fr/bukken/5...,仲介,(株)未来企画桜木町店,2023-09-12
4,100343709215,マリーナヨコハマベイサイド 203号室,2017年6月,神奈川県,横浜市,神奈川区東神奈川２,マンション,鉄筋コン,南東,5,2,1K,洋6.7,20.18,https://img01.suumo.com/front/gazo/fr/bukken/2...,仲介,(株)未来企画本店,2023-09-12


# Export

In [102]:
cost_info.to_csv('費用情報_処理後.csv',index=False,encoding='utf-8_sig')
facility_info.to_csv('設備情報_処理後.csv',index=False,encoding='utf-8_sig')
property_basic_info.to_csv('物件基本情報_処理後.csv',index=False,encoding='utf-8_sig')
property_info.to_csv('物件情報_処理後.csv',index=False,encoding='utf-8_sig')
